Sample database for MCU

In [1]:
import os
from dotenv import load_dotenv

#openai
import openai

# langchain
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

os.environ['NEO4J_USERNAME'] = 'neo4j'
os.environ['NEO4J_PS'] = 'UUMRX3vIec5aRPh4vT1Azy0KKNHK2KmkJFgiO1zqf6U'
os.environ['AURA_DB'] = '65fd0a31'

Connections

In [2]:
graph = Neo4jGraph(
    url=f"neo4j+s://{os.environ['AURA_DB']}.databases.neo4j.io",
    username=os.getenv('NEO4J_USERNAME'),
    password=os.getenv('NEO4J_PS')
)

Seed the database:
  - [Example](https://python.langchain.com/docs/use_cases/more/graph/graph_cypher_qa)

In [3]:
# X-men
graph.query(
    """
MERGE (f:Faction {name:"X-Men"})
WITH f
UNWIND ["Professor X", "Cyclops", "Iceman", "Beast", "Angel", "Marvel Girl"] AS hero
MERGE (h:Hero {name:hero})
MERGE (h)-[:MEMBER]->(f)
"""
)
#graph.refresh_schema()

# Avengers
graph.query(
    """
MERGE (f:Faction {name:"Avengers"})
WITH f
UNWIND ["Iron Man", "Captain America", "Hulk", "Thor", "Black Widow", "Hawkeye"] AS hero
MERGE (h:Hero {name:hero})
MERGE (h)-[:MEMBER]->(f)
"""
)
#graph.refresh_schema()

# Thor
graph.query(
    """
MERGE (f:Father {name:"Odin"})
WITH f
UNWIND ["Thor"] AS son
MERGE (s:Son {name:son})
MERGE (s)-[:SIRED_BY]->(f)
"""
)
#graph.refresh_schema()

# X-men enemies
graph.query(
    """
MERGE (f:Faction {name:"X-Men"})
WITH f
UNWIND ["Apocalypse", "Juggernaut", "Magneto", "Sabretooth", "Vulcan"] AS villian
MERGE (v:Villian {name:villian})
MERGE (f)-[:ENEMIES_WITH]->(v)
"""
)
#graph.refresh_schema()

# Avengers enemies
graph.query(
    """
MERGE (f:Faction {name:"Avengers"})
WITH f
UNWIND ["Blastaar", "Kang", "Magneto", "Thanos", "Ultron"] AS villian
MERGE (v:Villian {name:villian})
MERGE (f)-[:ENEMIES_WITH]->(v)
"""
)
graph.refresh_schema()

Examine the graph schema

In [4]:
print(graph.get_schema)


        Node properties are the following:
        [{'properties': [{'property': 'name', 'type': 'STRING'}], 'labels': 'Faction'}, {'properties': [{'property': 'name', 'type': 'STRING'}], 'labels': 'Hero'}, {'properties': [{'property': 'name', 'type': 'STRING'}], 'labels': 'Father'}, {'properties': [{'property': 'name', 'type': 'STRING'}], 'labels': 'Son'}, {'properties': [{'property': 'name', 'type': 'STRING'}], 'labels': 'Villian'}]
        Relationship properties are the following:
        []
        The relationships are the following:
        ['(:Faction)-[:ENEMIES_WITH]->(:Villian)', '(:Hero)-[:MEMBER]->(:Faction)', '(:Son)-[:SIRED_BY]->(:Father)']
        


Chat with the graph

In [5]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [6]:
chain.run('List the X-Men team')



> Entering new  chain...
Generated Cypher:
MATCH (h:Hero)-[:MEMBER]->(f:Faction {name: 'X-Men'})
RETURN h.name
Full Context:
[{'h.name': 'Marvel Girl'}, {'h.name': 'Angel'}, {'h.name': 'Beast'}, {'h.name': 'Iceman'}, {'h.name': 'Cyclops'}, {'h.name': 'Professor X'}]

> Finished chain.


'The X-Men team consists of Marvel Girl, Angel, Beast, Iceman, Cyclops, and Professor X.'

In [7]:
chain.run('List the Avengers team')



> Entering new  chain...
Generated Cypher:
MATCH (h:Hero)-[:MEMBER]->(f:Faction {name: 'Avengers'})
RETURN h.name
Full Context:
[{'h.name': 'Hawkeye'}, {'h.name': 'Black Widow'}, {'h.name': 'Thor'}, {'h.name': 'Hulk'}, {'h.name': 'Captain America'}, {'h.name': 'Iron Man'}]

> Finished chain.


'The Avengers team consists of Hawkeye, Black Widow, Thor, Hulk, Captain America, and Iron Man.'

In [8]:
chain.run('List the children of Odin')  # wrong!



> Entering new  chain...
Generated Cypher:
MATCH (father:Father {name: 'Odin'})-[:SIRED_BY]->(son:Son)
RETURN son.name
Full Context:
[]

> Finished chain.


'Odin, the Norse god, is said to have several children. Some of his well-known children include Thor, the god of thunder, and Loki, the trickster god. Additionally, Odin is also believed to have fathered Baldr, the god of light and purity, and Hodr, the blind god of darkness. These are just a few of the children associated with Odin in Norse mythology.'

In [11]:
chain.run('Who is the father of Thor?')



> Entering new  chain...
Generated Cypher:
MATCH (s:Son)-[:SIRED_BY]->(f:Father)
WHERE s.name = 'Thor'
RETURN f.name
Full Context:
[{'f.name': 'Odin'}]

> Finished chain.


'The father of Thor is Odin.'

In [12]:
chain.run('what villians do the X-Men and the Avengers have in common?')



> Entering new  chain...
Generated Cypher:
MATCH (xmen:Faction {name: 'X-Men'})-[:ENEMIES_WITH]->(villian:Villian)<-[:ENEMIES_WITH]-(avengers:Faction {name: 'Avengers'})
RETURN villian.name
Full Context:
[{'villian.name': 'Magneto'}]

> Finished chain.


'The X-Men and the Avengers have Magneto as a common villain.'

In [13]:
chain.run('What enemies do Thor and Iceman have in common?')



> Entering new  chain...
Generated Cypher:
MATCH (thor:Hero {name: 'Thor'})-[:MEMBER]->(faction:Faction)<-[:MEMBER]-(iceman:Hero {name: 'Iceman'})
MATCH (thor)-[:ENEMIES_WITH]->(enemy:Villian)<-[:ENEMIES_WITH]-(iceman)
RETURN enemy.name
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have any information on the enemies that Thor and Iceman have in common."

In [14]:
chain.run('What is the shortest path between Thor and Iceman?')



> Entering new  chain...
Generated Cypher:
MATCH (start:Hero {name: 'Thor'}), (end:Hero {name: 'Iceman'})
MATCH path = shortestPath((start)-[*]-(end))
RETURN path
Full Context:
[{'path': [{'name': 'Thor'}, 'MEMBER', {'name': 'Avengers'}, 'ENEMIES_WITH', {'name': 'Magneto'}, 'ENEMIES_WITH', {'name': 'X-Men'}, 'MEMBER', {'name': 'Iceman'}]}]

> Finished chain.


'The shortest path between Thor and Iceman is through the Avengers, who are enemies with Magneto, who in turn is enemies with the X-Men, and finally, Iceman is a member of the X-Men.'